

1.   Load  Raw_Skills_Dataset



In [ ]:
import pandas as pd

df1 = pd.read_csv('/content/data/Example_Technical_Skills.csv')

print(df1.head().to_string())

                       Technology Skills
0                    SAP Fiori Developer
1  Oracle Instance Management & Strategy
2           Boomi Master Data Management
3  Digital Manufacturing on Cloud ( DMC)
4                                 DevOps


Load Example_Technical_Skills

In [ ]:
df = pd.read_csv('/content/data/Raw_Skills_Dataset.csv')
df2 = pd.read_csv('/content/data/Raw_Skills_Dataset.csv')

print(df.head(10).to_string())

                   RAW DATA
0                  What ifs
1                 seniority
2               familiarity
3           functionalities
4                   Lambdas
5              Java Streams
6  Object Oriented analysis
7      Relational Databases
8                       SQL
9                       ORM


Install dependencies and Download needed files

In [ ]:
!pip install spacy==3.3
!python -m spacy download en_core_web_sm
!python -m spacy download en_core_web_lg
!pip install pyLDAvis
!pip install jsonlines

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.8 MB 5.5 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 400.7 MB 6.6 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#spacy a Natural Language Processing (NLP) libry
import spacy
from spacy.pipeline import EntityRuler #add our own Entity ruler
from spacy.lang.en import English   #we are working in english
from spacy.tokens import Doc  #Contains our text


nlp = spacy.load("en_core_web_lg")  #contains a large set of English words and 
                                    #for tokenisation


import pandas as pd #Data loading/ Data manipulation
import numpy as np

#nltk another Natural Language Processing (NLP) libry
import re
import nltk
from nltk.corpus import stopwords #it is not usefull for NLP Examples why, to, ...
from nltk.stem import WordNetLemmatizer #creates existing lemmer
nltk.download(['stopwords','wordnet']) #Downloading dictionary wordnet (for lemmer)
                                        # and stopwords

#warning
import warnings
warnings.filterwarnings('ignore')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Loading **jz_skill_patterns.jsonl**, a libry containing skills

In [ ]:
skill_pattern_path = "/content/data/jz_skill_patterns.jsonl"

Adding our custom Entity Ruler (jz_skill_patterns.jsonl) 

In [ ]:
ruler = nlp.add_pipe("entity_ruler")
ruler.from_disk(skill_pattern_path)
nlp.pipe_names

['tok2vec',
 'tagger',
 'parser',
 'attribute_ruler',
 'lemmatizer',
 'ner',
 'entity_ruler']

Preparing function for the Job. It will do Entity detection with skill detection, then we sellect only skills

In [ ]:
lm = WordNetLemmatizer()
def skills(text):
    doc = nlp(text)
    subset = []
    for ent in doc.ents:
        if ent.label_ == "SKILL":
            subset.append(ent.text)
    return subset


def unique_skills(x):
    return list(set(x))

Clean the Data to remove pontuations and stop words (english stop words) which will not help us here.

We lemmatize because same skill can be written in diffrent forms

In [ ]:
df2["Clean_Text"]=df["RAW DATA"].str.lower()

In [ ]:
df2["Clean_Text"] = df2["Clean_Text"].apply(lambda x: re.sub(
        '(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?"',
        " ",
        x,
    ).split())

In [ ]:
df2["Clean_Text"] = df2["Clean_Text"].apply(lambda x: ''.join([lm.lemmatize(word) for word in x if not word in stopwords.words("english")]))

Know get the skills. Now we suppose one cell can contain more than one technical skill, so we use a list of skills

In [ ]:
df2["skills"] = df2["Clean_Text"].apply(skills)


In [ ]:
df2["skills"] = df2["skills"].apply(lambda x: unique_skills(x))

In [ ]:
df2['skills'] = df2['skills'].apply(lambda y: np.nan if len(y)==0 else y) #convert empty list to Na
#Drop NAs to have only skills 
df2 = df2.dropna()

here We can either take the original words (RAW_DATA to have the Exemple form) or use SKILLS column




In [ ]:
#using SKILLS row
Skills = df2['skills'].apply(lambda x: ' '.join(x)).unique().tolist()

In [ ]:
#Using RAW_DATA row
Skills1 = list(df2['Clean_Text'].unique())

In [ ]:
Skills2 = list(df2['Clean_Text'].unique())

In [ ]:
Skills

['hibernate',
 'maven',
 'bamboo',
 'framework',
 'python',
 'mysql',
 'oracle',
 'celery',
 'azure',
 'grpc',
 'docker',
 'kubernetes',
 'vault',
 'database',
 'github',
 'ruby',
 'javascript',
 'algorithm',
 'angular',
 'meteor',
 'ember',
 'bootstrap',
 'typescript',
 'redux',
 'heap',
 'mongodb',
 'jasmine',
 'karma',
 'protractor',
 'junit',
 'django',
 'flask',
 'tensorflow',
 'deployment',
 'queue',
 'concurrency',
 'specification',
 'shell',
 'git',
 'xamarin',
 'devops',
 'sqlite',
 'documentation',
 'elasticsearch',
 'numpy',
 'pytorch',
 'debugging',
 'openshift',
 'bitbucket',
 'postgresql',
 'linux',
 'sinatra',
 'beanstalk',
 'rabbitmq',
 'rspec',
 'jest',
 'elixir',
 'graphql',
 'prometheus',
 'mocha',
 'grafana',
 'stackdriver',
 'scalability',
 'workflow',
 'certificate',
 'fastlane',
 'analytics',
 'lua',
 'ansible',
 'gulp',
 'gradle',
 'ethereum',
 'webpack',
 'schedule',
 'bash',
 'syntax',
 'hbase',
 'confluence',
 'cucumber',
 'airflow',
 'groovy',
 'gitlab',
 'm

In [ ]:
Skills1

['hibernate',
 'maven',
 'bamboo',
 'framework',
 'python',
 'mysql',
 'oracle',
 'celery',
 'azure',
 'grpc',
 'docker',
 'kubernetes',
 'vault',
 'database',
 'github',
 'ruby',
 'javascript',
 'algorithm',
 'angular',
 'meteor',
 'ember',
 'bootstrap',
 'typescript',
 'redux',
 'heap',
 'mongodb',
 'jasmine',
 'karma',
 'protractor',
 'junit',
 'django',
 'flask',
 'tensorflow',
 'deployment',
 'queue',
 'concurrency',
 'specification',
 'shell',
 'git',
 'xamarin',
 'devops',
 'sqlite',
 'documentation',
 'elasticsearch',
 'numpy',
 'pytorch',
 'debugging',
 'openshift',
 'bitbucket',
 'postgresql',
 'linux',
 'sinatra',
 'beanstalk',
 'rabbitmq',
 'rspec',
 'jest',
 'elixir',
 'graphql',
 'prometheus',
 'mocha',
 'grafana',
 'stackdriver',
 'scalability',
 'workflow',
 'certificate',
 'fastlane',
 'analytics',
 'lua',
 'ansible',
 'gulp',
 'gradle',
 'ethereum',
 'webpack',
 'schedule',
 'bash',
 'syntax',
 'hbase',
 'confluence',
 'cucumber',
 'airflow',
 'groovy',
 'gitlab',
 'm

In [ ]:
df2.head(20)

,RAW DATA,Clean_Text,skills
11,Hibernate,hibernate,[hibernate]
13,Hibernate,hibernate,[hibernate]
16,Maven,maven,[maven]
21,bamboo,bamboo,[bamboo]
23,frameworks,framework,[framework]
55,Python,python,[python]
62,MySQL,mysql,[mysql]
63,Oracle,oracle,[oracle]
70,Celery,celery,[celery]
77,Azure,azure,[azure]
